In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/california-housing-prices/housing.csv')

# Data exploration 
Done in the last kernel

# Drop noise examples
Case where the model should account amounts larger than 500 000  
Drop on test set also, so the model is not judged harshly for not predicting values larger than 500 000

In [ ]:
data = data[data['median_house_value'] < 500001]

# Data splitting
Should be done first, because some data transformers have to fit the data

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.20, random_state=42, stratify=None)
# Stratify the data: represent all price ranges in both datasets

In [ ]:
train_data_features = train_data.drop("median_house_value", axis=1)
train_data_target = train_data['median_house_value'].copy()

test_data_features = test_data.drop("median_house_value", axis=1)
test_data_target = test_data['median_house_value'].copy()

!! From now on only use the training dataset!!

# Fill missing values

In [ ]:
from sklearn.impute import SimpleImputer

train_data_numeric = train_data_features.drop(['ocean_proximity'], axis=1)

imputer = SimpleImputer(strategy="median")
imputer.fit(train_data_numeric)

In [ ]:
imputer.statistics_

In [ ]:
train_data_numeric.median().values

In [ ]:
train_data_clean = imputer.transform(train_data_numeric)

### Scikit Learn's Object interface
**Estimator:** object that can estimate some parameters based on a dataset
ex:  
- estimation done with the fit() method.  
dataset(X_train, (y_train)) --fit()--> learns the parameters (e.g. the median of a column) 

**Transformer:** estimator that can also transform a dataset  
- implement the transform() method  
dataset(X_train) --transform()--> transformed dataset(X_train_prepared)  

**Predictor:** estimator that can make a prediction given a dataset. 
- implements the predict() method.  
dataset of new instances(X_test) --predict()--> predictions. 
- implements a score() method  
test set(X_test, y_test) --score()--> quality score (r2, precision, accuracy, f1...) 

# Feature Engineering

In [ ]:
rooms_idx, bedrooms_idx, population_idx, household_idx = (
    list(train_data_features.columns).index(col) for col in ("total_rooms", "total_bedrooms", "population", "households")
)

In [ ]:
def add_feature_combinations(X, add_population_per_household=False, add_bedrooms_per_room=False):
    
    rooms_per_household = X[:, rooms_idx] / X[:, household_idx]
    # Add other two features
    
    return np.c_[X, rooms_per_household]

In [ ]:
from sklearn.preprocessing import FunctionTransformer

feature_adder = FunctionTransformer(add_feature_combinations, validate=False, kw_args={"add_population_per_household": False, "add_bedrooms_per_room": False})

train_data_features_engineered = feature_adder.fit_transform(train_data_clean)

# Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('feature_adder', FunctionTransformer(add_feature_combinations, validate=False)),
    ('std_scaler', StandardScaler())
])
categorical_pipeline = Pipeline([
    ('1hot', OneHotEncoder())
])

Pipeline:
- List of (name, Estimator) pairs
- All (but the last) estimators must be transformers
- Exposes the same methods as the final estimator

Calling Pipeline.fit()
- fit() method of Pipeline calls the fit_transform() method on all estimators squentially
- The output of each fit_transform() call is passed as a parameter to the next estimator
- The last estimator only has its fit() method called

Calling Pipeline.transform()
- only possible if the last estimator has a transform() method
- applies all transformations in a sequence

In [ ]:
train_data_features.columns

In [ ]:
from sklearn.compose import ColumnTransformer

num_columns = ['longitude', 'latitude', 'housing_median_age', 'total_rooms','total_bedrooms', 'population', 'households', 'median_income']
cat_columns = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ("num", numeric_pipeline, num_columns),
    ("cat", categorical_pipeline, cat_columns)
])

X_train = full_pipeline.fit_transform(train_data_features)

In [ ]:
y_train = train_data_target.values

# Select and train a model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Score and Evaluate a Model

In [ ]:
X_test = full_pipeline.transform(test_data_features)
y_test = test_data_target.values

In [ ]:
# Training error
lin_reg.score(X_train, y_train)

In [ ]:
# Generalization error
lin_reg.score(X_test, y_test)

Low Score! What could you do in general?  
- Select a more powerful model
- Feed the model with better features
- Reduce the constraints on the model (i.e. regularization)

# Try another Regression Model

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor()
sgd_reg.fit(X_train, y_train)

In [ ]:
sgd_reg.score(X_train, y_train)

In [ ]:
sgd_reg.score(X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)

In [ ]:
# Training error
tree_reg.score(X_train, y_train)

In [ ]:
# Generalization error
tree_reg.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)

In [ ]:
rf_reg.score(X_train, y_train)

In [ ]:
rf_reg.score(X_test, y_test)

Still overfitting:  
    - Regularize it
    - Get more training data

# Model tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2,3,4]}
]
rf_reg = RandomForestRegressor()

grid_search = GridSearchCV(rf_reg, param_grid, cv=5, scoring='r2', verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

In [ ]:
best_rf = grid_search.best_estimator_
best_rf

In [ ]:
# CV error
grid_search.best_score_

In [ ]:
# Generalization error
best_rf.score(X_test, y_test)

# Interpreting the model (not always possible)

In [ ]:
feature_importances = best_rf.feature_importances_
feature_importances

In [ ]:
num_features = num_columns
new_features = ["rooms_per_household"]
oneHot_features = full_pipeline.named_transformers_['cat']['1hot'].categories_[0].tolist()

features = num_features + new_features + oneHot_features

In [ ]:
sorted(zip(feature_importances, features), reverse=True)

# Save Your Model

In [ ]:
from joblib import dump

dump(best_rf, 'best_random_forest.joblib')